# Pancake Pridiction v2

In [1]:
from web3 import Web3
from web3.middleware import geth_poa_middleware
import datetime as dt
import pandas as pd
import numpy as n
import math

pd.set_option('display.max_rows', None)

In [2]:
import os
import sys

current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
sys.path.append(project_dir) if project_dir not in sys.path else None

import pancake
psp = pancake.Prediction()

2022-02-06 16:59:28.105 
  command:

    streamlit run C:\Users\Ehsan\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


### Config

In [3]:
dataset_reconds = 300

### Fetching Data

In [4]:
round_columns = ["epoch",
                 "startTimestamp",
                 "lockTimestamp",
                 "closeTimestamp",
                 "lockPrice",
                 "closePrice",
                 "lockOracleId",
                 "closeOracleId",
                 "totalAmount",
                 "bullAmount",
                 "bearAmount",
                 "rewardBaseCalAmount",
                 "rewardAmount",
                 "oracleCalled"]

def get_history(_psp, current_epoch, back_in_time=100):
    start_epoch_history = current_epoch - 2 - back_in_time

    df_history_round = pd.DataFrame(columns=round_columns)
    for i in range(start_epoch_history, current_epoch - 1):
        df_round = _psp.get_round(i)
        df_history_round = df_history_round.append(df_round)

    df_history_round = df_history_round.sort_values('epoch', ascending=False)
    df_history_round = df_history_round.reset_index(drop=True)
    
    return df_history_round

#### Creating a dataset

In [19]:
current_epoch = psp.get_current_epoch()
df = get_history(psp, current_epoch, dataset_reconds)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled
0,42961.0,1.644188e+09,1.644189e+09,1.644189e+09,412.729736,412.749939,0.0,0.0,36.186562,19.220308,16.966253,19.220308,35.100964,True
1,42960.0,1.644188e+09,1.644188e+09,1.644189e+09,411.601166,412.729736,0.0,0.0,15.624201,6.635990,8.988211,6.635990,15.155475,True
2,42959.0,1.644188e+09,1.644188e+09,1.644188e+09,411.459991,411.601166,0.0,0.0,26.103815,12.070806,14.033010,12.070806,25.320702,True
3,42958.0,1.644187e+09,1.644188e+09,1.644188e+09,412.472961,411.459991,0.0,0.0,20.628794,9.538275,11.090519,11.090519,20.009930,True
4,42957.0,1.644187e+09,1.644187e+09,1.644188e+09,412.165222,412.472961,0.0,0.0,21.725357,12.870497,8.854860,12.870497,21.073597,True


### Adding result

In [6]:
def add_result(lockPrice, closePrice):
    if lockPrice == closePrice:
        # draw
        return 0
    elif lockPrice > closePrice:
        # bear
        return -1
    elif lockPrice < closePrice:
        # bull
        return 1
        
df["result"] = df.apply(lambda row : add_result(row["lockPrice"], row["closePrice"]), axis=1)
df.head()

,epoch,startTimestamp,lockTimestamp,closeTimestamp,lockPrice,closePrice,lockOracleId,closeOracleId,totalAmount,bullAmount,bearAmount,rewardBaseCalAmount,rewardAmount,oracleCalled,result
0,42947.0,1.644184e+09,1.644184e+09,1.644185e+09,412.489990,412.735107,0.0,0.0,24.198120,11.717525,12.480595,11.717525,23.472178,True,1
1,42946.0,1.644184e+09,1.644184e+09,1.644184e+09,412.584900,412.489990,0.0,0.0,22.002338,10.668288,11.334050,11.334050,21.342268,True,-1
2,42945.0,1.644183e+09,1.644184e+09,1.644184e+09,412.833893,412.584900,0.0,0.0,21.718130,11.184771,10.533360,10.533360,21.066587,True,-1
3,42944.0,1.644183e+09,1.644183e+09,1.644184e+09,412.842194,412.833893,0.0,0.0,29.991182,15.489913,14.501270,14.501270,29.091448,True,-1
4,42943.0,1.644183e+09,1.644183e+09,1.644183e+09,412.545227,412.842194,0.0,0.0,22.453659,12.671292,9.782366,12.671292,21.780048,True,1


### Maximum Consequitive Occurance

In [7]:
countDf = pd.DataFrame({'result': list(df['result'][(df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(df['result'].groupby((df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

bear_count = countDf[countDf.result == -1]['runs'].max()
bull_count = countDf[countDf.result == 1]['runs'].max()
draw_count = countDf[countDf.result == 0]['runs'].max()

print(f"Bear {bear_count} / Bull {bull_count} / Draw {draw_count}")

Bear 7 / Bull 9 / Draw nan


## Strategies

In [9]:
running_columns = ["epoch", "position", "amount", "reward", "recent_loss", "factor"]
df_running = pd.DataFrame(columns=running_columns)

In [10]:
def bet(df_running, epoch, position, amount, recent_loss, factor):
    data = [epoch, position, amount, 0, recent_loss, factor]
    temp = pd.DataFrame(data=[data], columns=running_columns)
    df_running = df_running.append(temp)
    return df_running

    
def get_round_stats(df, epoch):
    df_round = df[df.epoch == epoch]

    total_amount = (df_round["bullAmount"] + df_round["bearAmount"]).iloc[0]
    if total_amount > 0:
        bull_ratio = ((df_round["bullAmount"] / total_amount) * 100).iloc[0]
        bear_ratio = ((df_round["bearAmount"] / total_amount) * 100).iloc[0]
        bear_pay_ratio = (total_amount / df_round["bearAmount"]).iloc[0]
        bull_pay_ratio = (total_amount / df_round["bullAmount"]).iloc[0]
    else:
        bull_ratio = None
        bear_ratio = None
        bear_pay_ratio = None
        bull_pay_ratio = None

    lockPrice = df_round["lockPrice"].iloc[0]
    closePrice = df_round["closePrice"].iloc[0]

    if lockPrice == closePrice:
        # draw
        result = 0
    elif lockPrice > closePrice:
        # bear
        result = -1
    elif lockPrice < closePrice:
        # bull
        result = 1
    
    return {"total_amount": total_amount,
           "bull_ratio": bull_ratio, "bear_ratio": bear_ratio,
           "bear_pay_ratio": bear_pay_ratio, "bull_pay_ratio": bull_pay_ratio,
           "result": result}


def update_reward(df, df_running, epoch, status):
    bet_value = df_running[df_running.epoch == epoch]["amount"].iloc[0]
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    
    if status == 0:
        df_running.loc[df_running.epoch == epoch, "reward"] = -1 * bet_value
        pay_ratio = 0
    elif status == 1:
        epoch_stats = get_round_stats(df, epoch)
        if bet_position == "bull":
            pay_ratio = epoch_stats["bull_pay_ratio"]
        elif bet_position == "bear":
            pay_ratio = epoch_stats["bear_pay_ratio"]

        df_running.loc[df_running.epoch == epoch, "reward"] = (bet_value * pay_ratio) - bet_value
    
    print(f"[{epoch}] - Factor {pay_ratio} - Bet {bet_value} - Position {position}")    
    return df_running


def check_result(df, df_running, epoch):
    
    bet_position = df_running[df_running.epoch == epoch]["position"].iloc[0]
    epoch_stats = get_round_stats(df, epoch)
    
    if bet_position == "bull" and epoch_stats["result"] == 1:
        result = 1
    elif bet_position == "bear" and epoch_stats["result"] == -1:
        result = 1
    else:
        result = 0
    
    df_running = update_reward(df, df_running, epoch, result)
    return df_running, epoch_stats["result"]
    

def overview(df_running):
    total_spent = df_running.sum()["amount"]
    total_loss = abs(df_running[df_running["reward"] < 0].sum()["reward"])
    loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    estimated_win = df_running[df_running["reward"] > 0].sum()["reward"]
    win_times = df_running[df_running["reward"] > 0].count()["reward"]
    estimated_gain = df_running.sum()["reward"]
    max_spent = df_running.max()["amount"]

    last_win_epoch = df_running[df_running["reward"] > 0].max()["epoch"]
    if last_win_epoch is None or math.isnan(last_win_epoch):
        recent_loss = abs(df_running.sum()["reward"])
        recent_loss_times = df_running[df_running["reward"] < 0].count()["reward"]
    else:
        recent_loss = abs(df_running[df_running["epoch"] > last_win_epoch].sum()["reward"])
        recent_loss_times = df_running[(df_running["epoch"] > last_win_epoch)
                                       & (df_running["reward"] < 0)].count()["reward"]

    return {"total_spent": total_spent,
            "max_spent": max_spent,
            "total_loss": total_loss,
            "loss_times": loss_times,
            "estimated_win": estimated_win,
            "win_times": win_times,
            "estimated_gain": estimated_gain,
            "recent_loss": recent_loss,
            "recent_loss_times": recent_loss_times}


### Strategy 1: Trend

### Strategy 2: Same-Before

### Strategy 3: Random

In [11]:
import random

starting_epoch = df.min()["epoch"] + 20
maximum_epoch = df.max()["epoch"] - 1

# 0: Even / 1: Odd
strategy_remainder = 0
base_bet = 0.001
safe_bet = 0.2

current_epoch = starting_epoch
value = base_bet
print("Starting Epoch: ", starting_epoch)

Starting Epoch:  42667.0


In [12]:
while current_epoch < maximum_epoch:
    if current_epoch % 2 == strategy_remainder:
    
        bet_status = overview(df_running)
        current_round_stats = get_round_stats(df, current_epoch)

        rand = random.getrandbits(1)

        if rand:
            # bull
            custom_factor = current_round_stats["bull_pay_ratio"] - safe_bet
            position = "bull"
        else:
            # bear
            custom_factor = current_round_stats["bear_pay_ratio"] - safe_bet
            position = "bear"

        value = (bet_status["recent_loss"] + base_bet) / (custom_factor - 1)
        if value < base_bet:
            value = base_bet

        df_running = bet(df_running, current_epoch, position, value, bet_status["recent_loss"], custom_factor)
        df_running, result = check_result(df, df_running, current_epoch)
        print(f"[{current_epoch}] - {position} - {value}")
    current_epoch += 1

[42668.0] - Factor 0 - Bet 0.0015940800901437502 - Position bull
[42668.0] - bull - 0.0015940800901437502
[42670.0] - Factor 0 - Bet 0.0021441913196966287 - Position bull
[42670.0] - bull - 0.0021441913196966287
[42672.0] - Factor 0 - Bet 0.009383388232892413 - Position bull
[42672.0] - bull - 0.009383388232892413
[42674.0] - Factor 0 - Bet 0.02219172274037721 - Position bear
[42674.0] - bear - 0.02219172274037721
[42676.0] - Factor 0 - Bet 0.0573603879142834 - Position bear
[42676.0] - bear - 0.0573603879142834
[42678.0] - Factor 0 - Bet 0.12675787710687395 - Position bull
[42678.0] - bull - 0.12675787710687395
[42680.0] - Factor 1.631900429725647 - Bet 0.5103760779869808 - Position bull
[42680.0] - bull - 0.5103760779869808
[42682.0] - Factor 1.6165903806686401 - Bet 0.002400439487812872 - Position bull
[42682.0] - bull - 0.002400439487812872
[42684.0] - Factor 2.399672508239746 - Bet 0.001 - Position bear
[42684.0] - bear - 0.001
[42686.0] - Factor 0 - Bet 0.0011522008545447183 - Po

### Overview

In [13]:
sum_reward = 0
df_running = df_running.reset_index(drop=True)

for index, row in df_running.iterrows():
        reward = row["reward"]
        sum_reward += reward
        
        df_running.loc[df_running.index == index, "acc_reward"] = sum_reward
        
df_running

,epoch,position,amount,reward,recent_loss,factor,acc_reward
0,42668.0,bull,0.001594,-0.001594,0.000000,1.627321,-0.001594
1,42670.0,bull,0.002144,-0.002144,0.001594,2.209817,-0.003738
2,42672.0,bull,0.009383,-0.009383,0.003738,1.504964,-0.013122
3,42674.0,bear,0.022192,-0.022192,0.013122,1.636348,-0.035313
4,42676.0,bear,0.057360,-0.05736,0.035313,1.633074,-0.092674
5,42678.0,bull,0.126758,-0.126758,0.092674,1.738998,-0.219432
6,42680.0,bull,0.510376,0.322507,0.219432,1.431900,0.103075
7,42682.0,bull,0.002400,0.00148,0.000000,1.416590,0.104555
8,42684.0,bear,0.001000,0.0014,0.000000,2.199673,0.105955
9,42686.0,bull,0.001152,-0.001152,0.000000,1.867904,0.104803


In [14]:
summary = overview(df_running)
print(summary)

{'total_spent': 2.229702804674718, 'max_spent': 0.5103760779869808, 'total_loss': 0.8104066492611639, 'loss_times': 75, 'estimated_win': 1.1593720800375549, 'win_times': 64, 'estimated_gain': 0.3489654307763907, 'recent_loss': 0.0, 'recent_loss_times': 0}


### Counting Consecutive Win/Loss

In [15]:
def add_result_binary(reward):
    if reward > 0:
        # bull
        return 1
    else:
        # bear
        return -1
        
temp_df = df_running.copy()

temp_df["result"] = temp_df.apply(lambda row : add_result_binary(row["reward"]), axis=1)

countDf = pd.DataFrame({'result': list(temp_df['result'][(temp_df['result'])
                       .diff().abs().fillna(1) > 0]),
                     'runs': list(temp_df['result'].groupby((temp_df['result'])
                       .diff().abs().fillna(1).cumsum()).count())})

loss_count = countDf[countDf.result == -1]['runs'].max()
win_count = countDf[countDf.result == 1]['runs'].max()

print(f"Loss {loss_count} / Win {win_count}")

Loss 8 / Win 7


In [43]:
import numpy as np

def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

currentPrice = np.round(df["closePrice"].iloc[0],1)
ema9 = np.round(calculate_ema(df["closePrice"].iloc[:9],9),1)
ema21 = np.round(calculate_ema(df["closePrice"].iloc[:21],21),1)

print('current price:', currentPrice)
print('ema9 :', ema9, ' ema21:',ema21)

if (ema9 == ema21):
    print('do nothing')

if (ema9 < ema21):
    if(currentPrice > ema9):
        print('bet bearish , confidence 100')
    else :
        print('bet bearish , confidence enough')
        
if (ema9 > ema21):
    if(currentPrice < ema9):
        print('bet bullish , confidence 100')
    else :
        print('bet bullish , confidence enough')
 


current price: 412.7
ema9 : [412.1]  ema21: [412.3]
bet bearish , confidence 100
